In [1]:
#import necessary packages
import pandas as pd
import numpy as np
import re

#read in data
data= pd.read_csv('dataanalystjobs.csv')
print(data.info())
print(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         2253 non-null   int64  
 1   Job Title          2253 non-null   object 
 2   Salary Estimate    2253 non-null   object 
 3   Job Description    2253 non-null   object 
 4   Rating             2253 non-null   float64
 5   Company Name       2252 non-null   object 
 6   Location           2253 non-null   object 
 7   Headquarters       2253 non-null   object 
 8   Size               2253 non-null   object 
 9   Founded            2253 non-null   int64  
 10  Type of ownership  2253 non-null   object 
 11  Industry           2253 non-null   object 
 12  Sector             2253 non-null   object 
 13  Revenue            2253 non-null   object 
 14  Competitors        2253 non-null   object 
 15  Easy Apply         2253 non-null   object 
dtypes: float64(1), int64(2),

**Cleaning: This Dataset is being cleaned to make it easier to work with in Tableau data visualisation software.

To Do:
- Remove second index row with no header
- Remove '(glassdoor)' string from 'salary est.' column
   - Remove ratings from 'company name' strings
- Remove 'easy apply', 'competitors', 'revenue', 'headquarters' columns
- Replace -1s with nans
- Split 'location' column into city, state columns
- Remove duplicates if there are any.
- Filter job titles into subcategories (entry level data analyst (jr. and junior), data analyst, senior data analyst,  business analyst, data scientist) using case when statement.

In [2]:
#remove second index row with no header
data = data.drop("Unnamed: 0",axis=1)

In [3]:
#Remove 'easy apply', 'competitors', 'revenue', 'headquarters'  columns
data= data[['Job Title','Salary Estimate','Job Description','Rating','Company Name','Location','Size','Founded','Type of ownership','Industry','Sector']]

In [4]:
#Split 'location' column into city, state columns
data[['City','State','Location']]= data['Location'].str.split(', ', expand=True, regex=True, n=2252)
data = data[['Job Title','Salary Estimate','Job Description','Rating','Company Name','City','State', 'Size','Founded','Type of ownership','Industry','Sector']]
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          2253 non-null   object 
 1   Salary Estimate    2253 non-null   object 
 2   Job Description    2253 non-null   object 
 3   Rating             2253 non-null   float64
 4   Company Name       2252 non-null   object 
 5   City               2253 non-null   object 
 6   State              2253 non-null   object 
 7   Size               2253 non-null   object 
 8   Founded            2253 non-null   int64  
 9   Type of ownership  2253 non-null   object 
 10  Industry           2253 non-null   object 
 11  Sector             2253 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 211.3+ KB
None


In [5]:
#remove ' (Glassdoor est.)' string from 'salary est.' column

# this was a bit complex as () have special significance within strings so you can simply remove them like other letters and numbers.  look at python notes for more details  
data['Salary Estimate']= data['Salary Estimate'].str.replace('Glassdoor est.','')
inputToken = ' ('
data['Salary Estimate']= data['Salary Estimate'].str.replace(re.escape(inputToken),'')
inputToken2 = ')'
data['Salary Estimate']= data['Salary Estimate'].str.replace(re.escape(inputToken2),'')

#Clean 'Type of ownership' column by compiling all data into 4 categories
#Remove 'n\3.4' ratings from Company Name column (public, private, gov, education, healthcare)
data['Company Name']= data['Company Name'].str[:-4]


#remove$ and K, split 'Estimate' column into 2 columns with high estimate/low estimate then take average of the two and convert it to an int datatype
data['Salary Estimate']= data['Salary Estimate'].str.replace('$','')
data['Salary Estimate']= data['Salary Estimate'].str.replace('K','000')
data['Salary Estimate']= data['Salary Estimate'].str.replace(' ','')
data[['Sal_estlow','Sal_esthigh']]= data['Salary Estimate'].str.split('-', expand=True, regex=True)

#drop row from datarfame that has null value for 'Salary Estimate'
data.drop(index=data.index[2149], 
        axis=0, 
        inplace=True)

#convert salary columns to 'int' data type
data['Sal_estlow']=data['Sal_estlow'].astype(int)
data['Sal_esthigh']=data['Sal_esthigh'].astype(int)

#calculate an average for salary using high and low estimate
data['AvgSal']= (data['Sal_esthigh']+data['Sal_estlow'])/2
data = data[['Job Title','AvgSal','Sal_estlow','Sal_esthigh', 'Salary Estimate','Job Description','Rating','Company Name','City','State', 'Size','Founded','Type of ownership','Industry','Sector']]
print(data)

                                              Job Title   AvgSal  Sal_estlow  \
0     Data Analyst, Center on Immigration and Justic...  51500.0       37000   
1                                  Quality Data Analyst  51500.0       37000   
2     Senior Data Analyst, Insights & Analytics Team...  51500.0       37000   
3                                          Data Analyst  51500.0       37000   
4                                Reporting Data Analyst  51500.0       37000   
...                                                 ...      ...         ...   
2248  RQS - IHHA - 201900004460 -1q Data Security An...  91000.0       78000   
2249              Senior Data Analyst (Corporate Audit)  91000.0       78000   
2250  Technical Business Analyst (SQL, Data analytic...  91000.0       78000   
2251                Data Analyst 3, Customer Experience  91000.0       78000   
2252                        Senior Quality Data Analyst  91000.0       78000   

      Sal_esthigh Salary Estimate  \
0 

/var/folders/ml/d3p4clqj3bj2z7m7hff04nsw0000gn/T/ipykernel_99984/2180278382.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Salary Estimate']= data['Salary Estimate'].str.replace('Glassdoor est.','')
/var/folders/ml/d3p4clqj3bj2z7m7hff04nsw0000gn/T/ipykernel_99984/2180278382.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Salary Estimate']= data['Salary Estimate'].str.replace(re.escape(inputToken),'')
/var/folders/ml/d3p4clqj3bj2z7m7hff04nsw0000gn/T/ipykernel_99984/2180278382.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Salary Estimate']= data['Salary Estimate'].str.replace(re.escape(inputToken2),'')
/var/folders/ml/d3p4clqj3bj2z7m7hff04nsw0000gn/T/ipykernel_99984/2180278382.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character re

In [6]:
#replace -1s with nans
data= data.replace('-1', np.nan)

data['Founded'] = data['Founded'].replace('-1', np.nan)

In [7]:
#remove duplicates if there are any
duplicates=data.duplicated(keep=False)
#there arent any duplicates as this command returns an empty dataframe

In [8]:
#filter job titles into subcategories (entry level data analyst (jr. and junior), data analyst, senior data analyst,  business analyst, data scientist) using case when statement.
juniordata= data[data['Job Title'].str.contains('junior|jr|entry level|intern', case=False)]
seniordata= data[data['Job Title'].str.contains('senior|chief|lead|principal|master|sr.', case=False)]


In [9]:
#remove commas from string columns and ad commas at beginning and end of columns.  THis will allow the data to be more easily parsed by BI software like Tableau
data['Job Title'] = data['Job Title'].str.replace(',','')

data['Job Description'] = data['Job Description'].str.replace(',','')
data['Job Description'] = data['Job Description'].str.replace(',','')
data['Company Name'] = data['Company Name'].str.replace(',','')
data['Size'] = data['Size'].str.replace(',','')
data['Type of ownership'] = data['Type of ownership'].str.replace(',','')
data['Industry'] = data['Industry'].str.replace(',','')
data['Sector'] = data['Sector'].str.replace(',','')
print(data)

                                              Job Title   AvgSal  Sal_estlow  \
0     Data Analyst Center on Immigration and Justice...  51500.0       37000   
1                                  Quality Data Analyst  51500.0       37000   
2     Senior Data Analyst Insights & Analytics Team ...  51500.0       37000   
3                                          Data Analyst  51500.0       37000   
4                                Reporting Data Analyst  51500.0       37000   
...                                                 ...      ...         ...   
2248  RQS - IHHA - 201900004460 -1q Data Security An...  91000.0       78000   
2249              Senior Data Analyst (Corporate Audit)  91000.0       78000   
2250  Technical Business Analyst (SQL Data analytics...  91000.0       78000   
2251                 Data Analyst 3 Customer Experience  91000.0       78000   
2252                        Senior Quality Data Analyst  91000.0       78000   

      Sal_esthigh Salary Estimate  \
0 

In [10]:
#double check cleaned df's and export them as .csv
data.to_csv('datajobscleaned.csv', na_rep='NaN', index=False, sep=',')

juniordata.to_csv('juniordatajobscleaned.csv', na_rep='NaN', index=False, sep=',')
seniordata.to_csv('seniordatajobscleaned.csv', na_rep='NaN', index=False, sep=',')